<a href="https://colab.research.google.com/github/eric88525/DRCD/blob/master/trainDRCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRCD

In [1]:
!nvidia-smi

Fri Aug  7 15:49:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 該來用TPU了吧
+ [REF](https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb#scrollTo=H9KYz-Vk4fMa)

In [2]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [3]:
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  57471      0 --:--:-- --:--:-- --:--:-- 57471
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
     |████████████████████████████████| 61kB 2.6MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     


In [3]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

In [4]:
# Creates a random tensor on xla:1 (a Cloud TPU core)
device = xm.xla_device()
t1 = torch.ones(3, 3, device = device)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:1')


# Setup

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [1]:
!pip install transformers
!pip install opencc
!pip install pyprind
!pip install zhon

KeyboardInterrupt: ignored

In [1]:
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
import json
from time import strftime,gmtime
from opencc import OpenCC
import pyprind
from sklearn.utils import shuffle
import re
from zhon.hanzi import non_stops,stops
import numpy as np
import random
import argparse
import time
from datetime import datetime, timedelta

print(torch.cuda.is_available())

True


In [2]:
#cd ./drive/My Drive/Colab Notebooks/DRCD

In [3]:
#!ls

# Get Train Dev Test

+ train 8014 26936
+ test 1000 3493
+ dev 1000 3524
  

In [2]:
import json

class DRCDdataset():
  def __init__(self,train_path=None , test_path = None ,dev_path = None):
    
    if train_path != None:
      self.train = self.get_data(train_path)
    if test_path != None:
      self.test = self.get_data(test_path)
    if dev_path != None:
      self.dev = self.get_data(dev_path)

  def get_data(self,path):
    input_file = open(path)
    ds = json.load(input_file)
    datas = []
    for i in range(len(ds['data'])):
      for j in ds['data'][i]['paragraphs']:
        context = j['context']
        question = [q['question'] for q in j['qas']]
        ans = [ a['answers'][0]['text'] for a in j['qas'] ]
        #ans = [a['answers'][0]['answer_start'] for a in j['qas']]
        #anse = [a['answers'][0]['answer_start']+len(a['answers'][0]['text'])-1 for a in j['qas']]
        for k in range(len(question)):
          datas.append([context,question[k],ans[k]])
          #datas.append([context,question[k],ans[k],anse[k]])
    return datas


# Dataset

In [3]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class GetDataset(Dataset):
    def __init__(self,data,model_type,device,language):
      self.data = data
      self.tokenizer = BertTokenizer.from_pretrained(model_type)
      self.device = device
      self.cc = OpenCC('t2s') # tw->china
      self.language = language
    def __getitem__(self,idx):
      paragraph,question,ans = self.data[idx][0] , self.data[idx][1], self.data[idx][2] 
      if self.language == 'china':
        paragraph,question,ans = self.cc.convert(paragraph),self.cc.convert(question),self.cc.convert(ans)
        
      token_tensor = self.tokenizer.encode_plus(question,paragraph,max_length=512,truncation=True,pad_to_max_length=True)
      #print(paragraph[:20],question,ans)
      s_idx = [0]*512
      e_idx = [0]*512
      # 答案tok
      s_tok = self.tokenizer.encode(ans)[1:-1]
      # 找到開頭跟他一樣的
      s_lsit = [i for i, x in enumerate(token_tensor['input_ids']) if x == s_tok[0]]
      
      for s_pos in s_lsit:
        e_pos = s_pos+len(s_tok)
        if e_pos > 511:
          continue
          
        if token_tensor['input_ids'][s_pos:e_pos] == s_tok:
          s_idx[s_pos] = 1
          e_idx[e_pos-1] = 1
          break
      s_tensor = torch.Tensor(s_idx)
      e_tensor = torch.Tensor(e_idx)
      # input_ids / token_type_ids / attention_mask / s_tensor / e_tensor
      return {'input_ids': torch.tensor(token_tensor['input_ids']).to(self.device)
          ,'token_type_ids': torch.tensor( token_tensor['token_type_ids']).to(self.device) 
          ,'attention_mask': torch.tensor( token_tensor['attention_mask']).to(self.device)
          ,'s_tensor': s_tensor.to(self.device)
          ,'e_tensor': e_tensor.to(self.device)}
    def __len__(self):
      return len(self.data)
    

In [4]:
"""
x = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')
print(len(x.train),len(x.test),len(x.dev))
device = torch.device('cuda:0')
model_type = 'hfl/chinese-roberta-wwm-ext'
language = 'china'
z = GetDataset(x.dev,model_type,device,language)
tokenizer = BertTokenizer.from_pretrained(model_type)
i = 10

tok = tokenizer.convert_ids_to_tokens(z[i]['input_ids'])
s = z[i]['s_tensor'].tolist()
e = z[i]['e_tensor'].tolist()
tp = z[i]['token_type_ids'].tolist()
at = z[i]['attention_mask'].tolist()
for i in range(len(tok)):
  print(tok[i],s[i],e[i],tp[i],at[i])
"""

"\nx = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')\nprint(len(x.train),len(x.test),len(x.dev))\ndevice = torch.device('cuda:0')\nmodel_type = 'hfl/chinese-roberta-wwm-ext'\nlanguage = 'china'\nz = GetDataset(x.dev,model_type,device,language)\ntokenizer = BertTokenizer.from_pretrained(model_type)\ni = 10\n\ntok = tokenizer.convert_ids_to_tokens(z[i]['input_ids'])\ns = z[i]['s_tensor'].tolist()\ne = z[i]['e_tensor'].tolist()\ntp = z[i]['token_type_ids'].tolist()\nat = z[i]['attention_mask'].tolist()\nfor i in range(len(tok)):\n  print(tok[i],s[i],e[i],tp[i],at[i])\n"

# Model

In [4]:
class bertDRCD(nn.Module):
    def __init__(self,model_type):
        super(bertDRCD,self).__init__()

        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
        self.bert_model = BertModel.from_pretrained(model_type,config = config)
  
        #self.s_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 
        #self.e_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 

        self.s_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,1)
        ) 
        self.e_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,1)
        )
        #self.start()

    def start(self):
           
      nn.init.xavier_uniform_(self.s_decoder[0].weight)
      nn.init.constant_(self.s_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.s_decoder[3].weight)
      nn.init.constant_(self.s_decoder[3].bias, 0)
      nn.init.xavier_uniform_(self.e_decoder[0].weight)
      nn.init.constant_(self.e_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.e_decoder[3].weight)
      nn.init.constant_(self.e_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None): 
      # hidden (batch,seq_len,hidden)
      hidden = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[0]
      s = self.s_decoder(hidden).squeeze()
      M = token_type_ids.clone().float().to(hidden.device).detach()
      M[M != 1] = float('-inf')
      s = s+M 
      #s = torch.softmax(s,dim=-1)
      e = self.e_decoder(hidden).squeeze()
      e = e + M
      #e = torch.softmax(e,dim=-1)
      return s,e

In [7]:
"""
class bertDRCD(nn.Module):
    def __init__(self,model_type):
        super(bertDRCD,self).__init__()

        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
        self.bert_model = BertModel.from_pretrained(model_type,config = config)
  
        self.s_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 
        self.e_vector = nn.Parameter(torch.randn(config.hidden_size),requires_grad=True) 

        #self.start()

    def start(self):
      self.decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,1)
      )     
      nn.init.xavier_uniform_(self.s_decoder[0].weight)
      nn.init.constant_(self.s_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.s_decoder[3].weight)
      nn.init.constant_(self.s_decoder[3].bias, 0)
      nn.init.xavier_uniform_(self.e_decoder[0].weight)
      nn.init.constant_(self.e_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.e_decoder[3].weight)
      nn.init.constant_(self.e_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None): 
      # hidden (batch,seq_len,hidden)
      hidden = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[0]
      s = torch.sum(hidden*self.s_vector,-1) # s now is (batch,seq_len)
      M = token_type_ids.clone().float().to(hidden.device).detach()
      M[M != 1] = float('-inf')
      s = s+M 
      #s = torch.softmax(s,dim=-1)
      e = torch.sum(hidden*self.e_vector,-1)
      e = e + M
      #e = torch.softmax(e,dim=-1)
      return s,e
"""

In [8]:
"""
device = torch.device('cpu')
device = torch.device('cuda:0')

model_type = 'bert-base-chinese'
model = bertDRCD(model_type).to(device)
ds = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')
dataset = GetDataset(ds.dev,model_type,device)
trainLoader = DataLoader(dataset, batch_size=8  ,shuffle=True)
criterion = nn.BCELoss()

for i , batch in enumerate(trainLoader):
  #print(batch)
  inp_ids,tok_id,att_m ,slabel,elabel = batch['input_ids'],batch['token_type_ids'],batch['attention_mask'],batch['s_tensor'],batch['e_tensor']
  s,e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)
  print(s)
  print(e)
  loss = (criterion(s,slabel) + criterion(e,elabel)) / 2
  print(loss)
  
  break
"""

"\ndevice = torch.device('cpu')\ndevice = torch.device('cuda:0')\n\nmodel_type = 'bert-base-chinese'\nmodel = bertDRCD(model_type).to(device)\nds = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')\ndataset = GetDataset(ds.dev,model_type,device)\ntrainLoader = DataLoader(dataset, batch_size=8  ,shuffle=True)\ncriterion = nn.BCELoss()\n\nfor i , batch in enumerate(trainLoader):\n  #print(batch)\n  inp_ids,tok_id,att_m ,slabel,elabel = batch['input_ids'],batch['token_type_ids'],batch['attention_mask'],batch['s_tensor'],batch['e_tensor']\n  s,e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)\n  print(s)\n  print(e)\n  loss = (criterion(s,slabel) + criterion(e,elabel)) / 2\n  print(loss)\n  \n  break\n"

# Test

In [5]:
def test(model,ds,args,tp):
    if tp == 'test':
        dataset = GetDataset(ds.test,args.model_type,args.device,args.language)
        print(f'Testing...{len(dataset)}')
    elif tp == 'dev':   
        dataset = GetDataset(ds.dev,args.model_type,args.device,args.language)
        print(f'Dev...{len(dataset)}')
    loader = DataLoader(dataset,batch_size=args.batch_size,shuffle=True)
    model.eval()
    loss,i = 0,0
    print(loss,i)
    criterion = nn.BCELoss()
    with torch.no_grad():
        for batch in loader:
            i = i+1
            inp_ids,tok_id,att_m ,slabel,elabel = batch['input_ids'],batch['token_type_ids'],batch['attention_mask'],batch['s_tensor'],batch['e_tensor']
            s,e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)
            #print(f's {s} \n e {e} \n sl {torch.softmax(s,dim=-1)} \n el {torch.softmax(e,dim=-1)} \n ')
            #print(f'SL {slabel} \n EL {elabel}')
            batch_loss = (criterion(torch.softmax(s,dim=-1),slabel) + criterion(torch.softmax(e,dim=-1),elabel)) / 2
            loss += batch_loss
            
        print(f'Result: LOSS: {loss} AVG: {loss/i} ')
    return loss

# Train
+ train(ds,w_d,lr_rate,device,model_type,epoches):
+ data(還沒封裝) , weight_d , learningrate , device , model_type , epoches

In [7]:
def train(ds,args):
    dataset = GetDataset(ds.train,args.model_type,args.device,args.language)
    trainLoader = DataLoader(dataset,batch_size=args.batch_size,shuffle=True)
    model = bertDRCD(args.model_type).to(args.device)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = AdamW(parameters, lr=args.learning_rate, weight_decay=args.weight_decay)
    criterion = nn.BCELoss()
    model.train()
    minloss = 100000
    for ei in range(args.epoch):
        model.train()
        epoch_loss,i,check_loss = 0,0,0
        for batch in trainLoader:
            i+=1
            inp_ids,tok_id,att_m ,slabel,elabel = batch['input_ids'],batch['token_type_ids'],batch['attention_mask'],batch['s_tensor'],batch['e_tensor']
            s,e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)
            #print(f's {s} \n e {e} \n sl {torch.softmax(s,dim=-1)} \n el {torch.softmax(e,dim=-1)} \n ')
            #print(f'SL {slabel} \n EL {elabel}')
            batch_loss = (criterion(torch.softmax(s,dim=-1),slabel) + criterion(torch.softmax(e,dim=-1),elabel)) / 2
            epoch_loss += batch_loss
            check_loss += batch_loss
            #print(batch_loss)
            batch_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if i % 1000==0:
                print(f'1000 batch LOSS {check_loss}')
                check_loss = 0
        print(f'Epoches: {ei} Loss {epoch_loss} ')
        dev_loss = test(model,ds,args,'dev') 
        if dev_loss < minloss:
            minloss = dev_loss
            best_model = copy.deepcopy(model.state_dict())
            
    return best_model

# Main

In [11]:
parser = argparse.ArgumentParser([])
parser.add_argument('--batch-size', default=4 , type=int)
parser.add_argument('--epoch', default=5, type=int)
parser.add_argument('--learning-rate', default=1e-5, type=float)    
parser.add_argument('--weight-decay', default=0.001, type=float)
parser.add_argument('--model-type', default='hfl/chinese-roberta-wwm-ext' , type=str)  #model_type = 'hfl/chinese-bert-wwm'  'hfl/chinese-roberta-wwm-ext'  'hfl/chinese-roberta-wwm-ext-large'
parser.add_argument('--device', default=torch.device('cuda:0'), type=int)
parser.add_argument('--language', default='tw', type=str)
args = parser.parse_args([])
args

Namespace(batch_size=4, device=device(type='cuda', index=0), epoch=5, language='tw', learning_rate=1e-05, model_type='hfl/chinese-roberta-wwm-ext', weight_decay=0.001)

In [9]:
ds = DRCDdataset(train_path='./dataset/DRCD_train.json',test_path='./dataset/DRCD_test.json',dev_path='./dataset/DRCD_dev.json')

In [10]:
ds.train[:1]

[['2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
  '廣州的快速公交運輸系統每多久就會有一輛巴士？',
  '10秒鐘']]

In [12]:
mode = 'train'

if mode == 'train': 
    print('Train')
    best_model = train(ds,args)
    if not os.path.exists('saved_models'):
        os.makedirs('saved_models')    
    modelname = 'bertDRCD'+'_'+(datetime.now()+timedelta(hours=8)).strftime("%m%d_%H%M")+'.pt' 
    torch.save(best_model, f'saved_models/{modelname}')
    print(f'Train end, model name is {modelname}')

elif mode == 'test' or mode == 'dev':
    modelname = 'bertDRCD_0808_1213.pt'
    test_model = bertDRCD(args.model_type).to(args.device)
    test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
    test(test_model,ds,args,mode)

#22.53021812438965
#22.579822540283203
# Result: LOSS: 3.2912609577178955 AVG: 0.003765744622796774 
# Result: LOSS: 3.526034116744995 AVG: 0.004002308938652277 

Train
1000 batch LOSS 8.626931190490723
1000 batch LOSS 4.643729209899902
1000 batch LOSS 4.884631633758545
1000 batch LOSS 3.8254661560058594
1000 batch LOSS 4.048442840576172
1000 batch LOSS 4.460489749908447
Epoches: 0 Loss 33.80564880371094 
Dev...3524
0 0
Result: LOSS: 3.2324342727661133 AVG: 0.0036690514534711838 
1000 batch LOSS 3.6819632053375244
1000 batch LOSS 2.970808744430542
1000 batch LOSS 3.835580825805664
1000 batch LOSS 3.925830364227295
1000 batch LOSS 3.231854200363159
1000 batch LOSS 3.097957134246826
Epoches: 1 Loss 23.001644134521484 
Dev...3524
0 0
Result: LOSS: 3.1757328510284424 AVG: 0.003604691242799163 
1000 batch LOSS 3.025343418121338
1000 batch LOSS 2.7990355491638184
1000 batch LOSS 3.188946485519409
1000 batch LOSS 3.041139841079712
1000 batch LOSS 2.3215901851654053
1000 batch LOSS 2.741631269454956
Epoches: 2 Loss 19.335426330566406 
Dev...3524
0 0
Result: LOSS: 3.1719062328338623 AVG: 0.0036003477871418 
1000 batch LOSS 2.327446460723877
1000 batch LO

# Package

In [15]:
class DRCD():
    def __init__(self,model_path,model_type,language): 
        # device
        # language define the language model wanna eat
        self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
        # model & tokenizer
        # model_type = 'hfl/chinese-roberta-wwm-ext'
        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
        self.tokenizer = BertTokenizer.from_pretrained(model_type)
        self.model = bertDRCD(model_type).to(self.device)
        self.model.load_state_dict(torch.load(model_path)) 
        self.language = language
        # 繁簡轉換
        self.c2tw = OpenCC('s2t') # china to tw
        self.tw2c = OpenCC('t2s') # tw to china

    def process(self,content,question):
        
        content,question = self.clean_str(content,question)
        
        
        with torch.no_grad():
            
            token_tensor = self.tokenizer.encode_plus(str(question),str(content),max_length=512,truncation=True,pad_to_max_length=True)
            token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
            segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
            mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
            answer_start,answer_end = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment) 
      
            # get ans
            tokens = self.tokenizer.convert_ids_to_tokens(token.squeeze())
            answer_start = answer_start.argmax(1)
            answer_end = answer_end.argmax(1)
            #print(answer_start,answer_end)
            if answer_start > answer_end :
                return '抱歉 這題有點難耶'
            
            #print(answer_start,answer_end)
            answer = ''.join(tokens[answer_start:answer_end+1])
            
            if self.language == 'tw':
                answer = self.c2tw.convert(answer)
            
            
        
        return answer
    
    def clean_str(self,content,question):
        content = content.replace(' ','')
        question = question.replace(' ','')
        if self.language == 'china':
            content = self.tw2c.convert(content)
            question = self.tw2c.convert(question)

        return content,question

In [16]:
modelname = 'bertDRCD_0810_0518.pt'
#myfriend = DRCD(f'saved_models/{modelname}','hfl/chinese-roberta-wwm-ext','tw')

In [17]:

#content = '福茂唱片音樂股份有限公司是一家臺灣唱片公司，成立於1961年，由福茂工程創辦人張人鳳及次子張耕宇先生創立。曾為環球音樂旗下子公司之一。至2002年，正式退出環球音樂旗下，成為獨立唱片公司。成立初期，福茂為英國迪卡唱片在台灣的獨家代理公司，1986年成立國語部，開始發展國語唱片市場，庾澄慶為第一個與福茂唱片簽約的台灣歌手。其後福茂一手捧紅歌手包括庾澄慶、邰正宵、辛曉琪、林隆璇和范曉萱等人。1989年福茂唱片引進CD技術，發行的首張CD專輯是庾澄慶的《讓我一次愛個夠》，銷量40萬張，並入選台灣百佳唱片第48位。1992年，迪卡唱片母公司寶麗金唱片向福茂買入60%股權，福茂也同時加入寶麗金唱片旗下；福茂英文名稱因此改為「Decca Records Taiwan Ltd.」並使用迪卡唱片商標。首張以迪卡唱片商標發行的唱片是庾澄慶首張英文專輯《哈林音樂電台》。在此時期，福茂一手捧紅大陸歌手那英和香港歌手周慧敏、蘇永康、王菲等在台灣的國語市場。1999年，寶麗金唱片集團被現時的環球唱片收購，福茂也曾為環球旗下子公司之一。'
content = '2014年世界盃外圍賽，韓國在首輪分組賽以首名出線次輪分組賽，與伊朗、卡達、烏茲別克以及黎巴嫩爭逐兩個直接出線決賽周資格，最後韓國僅以較佳的得失球差壓倒烏茲別克，以小組次名取得2014年世界盃決賽周參賽資格，也是韓國連續八次晉身世界盃決賽周。可惜南韓在決賽周表現不濟，三戰一和兩負小組末席出局。2018年世界盃外圍賽，韓國再次在首輪分組賽以首名出線次輪分組賽，再與伊朗、卡達、烏茲別克同組，同組還有中國及敘利亞。最後韓國以兩分壓倒敘利亞及烏茲別克，再以小組次名取得2018年世界盃決賽周參賽資格，也是韓國連續九次晉身世界盃決賽周。韓國的世界盃成績雖然是亞洲最佳，但在亞洲盃足球賽成績就遠不如世界盃。韓國除了在首兩屆亞洲杯奪冠外，但之後一直與亞洲盃錦標無緣，自1992年至2011年更連續六屆未能打入過亞洲盃決賽。2015年亞洲盃足球賽，韓國以五連勝一球不失的姿態，廿七年來首次打入亞洲盃決賽，對手是東道主澳洲。雖然韓國在分組初賽曾以1-0擊敗澳洲，但這場決賽韓國卻先失一球，最後在下半場補時階段扳平，令比賽進入加時階段，可惜澳洲最後在加時階段攻入致勝一球，最後韓國以1-2敗陣，只得亞軍。'
question = "哪一個國家負責舉辦2015年亞洲盃足球賽?"
ans = myfriend.process(content,question)
ans

NameError: name 'myfriend' is not defined

In [4]:
content = \
'美國國家情報總監辦公室7日發布新聞稿，分析外國影響美國總統選舉的最新情勢。美方評估，北京認為美國總統川普「無法預測」，希望川普不要贏得連任。川普說，若拜登當總統，則「中國將擁有我們這個國家。」\
川普7日晚間在紐澤西的鄉村俱樂部舉行記者會，被問到此事說，中國大陸、俄羅斯和伊朗都不想看到他贏得選舉，「若拜登當總統，則中國將擁有我們這個國家。」他也暗示，這三個國家可能透過郵寄投票，在美國大選中作弊。\
川普稱自己已聽取這分簡報，但似乎沒搞清楚內容，記者詢問「俄羅斯正在傷害拜登、而中國想看到你輸，你相信這分情報嗎？」\
川普說，可能是真的、可能是假的，實際上他做了很多事，包括醫療物資的儲備系統和軍隊，其中促使北約(NATO)各國提高軍事預算，用來抵抗俄羅斯，「沒人像我對俄羅斯這麼強硬，俄羅斯最不想看到我當選。」\
川普說，中國大陸做夢都想看到拜登打敗他成為總統，「若拜登是總統，則中國會擁有我們這個國家。」\
當記者試圖糾正川普，他接著說，「你沒把報告看清楚吧，報告裡還有個伊朗，伊朗也不想看到我當總統，但我要說，若我贏得連任，我會很快與伊朗、與北韓達成協議；若不是我2016年當選總統，可能美朝之間早已開戰。」\
川普說，若拜登主政下的美國，與中國大陸達成任何協議，則中國大陸會擁有美國；他則可以從中國大陸手中，拿到數百億美元，且美國現在一片欣欣向榮，各種指數表現良好。\
川普說，中國大陸、俄羅斯和伊朗都不想看到他連任，但是最大威脅不是這三個國家，是郵寄投票，包括這些國家甚至北韓等國，都可以在這裡作弊，他會仔細留意這個大問題。'

print(len(content))

question = ['蒂埃里·亨利是誰','蒂埃里·亨利為法國隊出場幾次','亨利曾在國際比賽中上演過帽子戲法嗎?',
            '蒂埃里·亨利是誰在哪年歐洲國家盃外圍賽中曾單場上演大四喜?','亨利在熱身賽中共打進多少球?']
"""
for q in question:
    print(q + ':  '+ myfriend.process(content,q))
"""

654


"\nfor q in question:\n    print(q + ':  '+ myfriend.process(content,q))\n"

In [10]:
content = \
'蒂埃里·亨利是前法國足球運動員。在他職業生涯參加的國際賽事中，他為法國隊出場123次，\
打進51球。他的首個國際比賽進球是在1998年世界盃對陣南非的比賽中。截至2015年10月，他是法國隊的頭號射手\
。2007年10月，他在對陣立陶宛的比賽中打進兩球，打破了米歇爾·普拉蒂尼42球的法國隊進球紀錄。亨利於2010年7月正式退役\
。亨利在2009年10月對奧地利的比賽中打進了個人國家隊第51粒進球，這也是他的最後一粒進球\
。亨利從未在國際比賽中上演過帽子戲法，儘管他曾7次在單場比賽梅開二度。\
他在對陣馬爾他的比賽中進球最多，在2004年歐洲國家盃外圍賽中曾單場上演大四喜。\
亨利一半以上的進球來自於主場比賽，他的51個進球中有31個是在法國本土打進，\
其中有20個在法蘭西體育場。亨利在熱身賽中共打進16球。在2003年國際足總洲際國家盃上\
，亨利打進四球並榮膺最佳射手，他也因此被評為「賽事最傑出球員」。\
亨利在歐洲國家盃外圍賽中打入12球，其中在2004年歐洲杯外圍賽打進6球，他最終排在射手榜第三位。'

question = ['蒂埃里·亨利是誰','蒂埃里·亨利為法國隊出場幾次','亨利曾在國際比賽中上演過帽子戲法嗎?',
            '蒂埃里·亨利是誰在哪年歐洲國家盃外圍賽中曾單場上演大四喜?','亨利在熱身賽中共打進多少球?']

for q in question:
    print(q + ':  '+ myfriend.process(content,q))

蒂埃里·亨利是誰:  前法國足球運動員
蒂埃里·亨利為法國隊出場幾次:  123次
亨利曾在國際比賽中上演過帽子戲法嗎?:  從未
蒂埃里·亨利是誰在哪年歐洲國家盃外圍賽中曾單場上演大四喜?:  2004年
亨利在熱身賽中共打進多少球?:  16球


In [ ]:
myfriend = DRCD(f'saved_models/{modelname}','hfl/chinese-roberta-wwm-ext','tw')

In [6]:


content = '2020年4月17日，高雄市選舉委員會公告罷免有效聯署書達到37.7萬，罷韓第二階段通過\
，韓國瑜成為全國第一位被罷免投票兩次者、也成為首個市長罷免案進入投票階段的市長。2020年6月6日\
，罷免投票通過，6月12日中央選舉委員會公告結果後正式解職\
，韓國瑜成為中華民國選舉史上首位被罷免成功的縣市首長。\
韓國瑜曾自述，他在年少求學階段考試都考第一名，但國中二年級開始因青春期懵懂，\
每天上課一直看前排女同學的小腿，成績一落千丈，國中三年級被轉入放牛班，逃學翹課\
、打撞球、打架也樣樣俱全，最後在父母決定下，18歲時入中華民國陸軍軍官學校專修學生班40期\
，軍銜升至上尉，自稱在軍中受一位國立臺灣大學醫學系畢業的預官轉魔術方塊的啟發，感到自己的不足\
，決定重拾課本。退伍前一年考上東吳大學英國語文學系，\
畢業後又考取國立政治大學東亞研究所和淡江大學國際事務與戰略研究所。\
由於政大免學費還發獎學金故選擇政大東亞所就讀。就讀期間擔任臺北市議員馮定國的助理，\
畢業後獲法學碩士學位。碩士論文名稱為《從中共「對臺統戰」策略看兩航談判》，指導教授為蘇起。'

question = ['罷免韓國瑜第二階段有通過嗎','中華民國選舉史上首位被罷免成功的縣市首長是誰'
            ,'罷免有效聯署書達到多少?','韓國瑜在哪天正式解職','韓國瑜幾歲被轉入放牛班'
            ,'韓國瑜每天上課一直看誰小腿','韓國瑜的碩士論文名稱?',
            '韓國瑜退伍前一年考上什麼','韓國瑜為什麼成績一落千丈','韓國瑜擔任誰的助理']

for q in question:
    print(q + ':  '+ myfriend.process(content,q))

罷免韓國瑜第二階段有通過嗎:  2020年4月17日，高雄市選舉委員會公告罷免有效聯署書達到37.7萬，罷韓第二階段通過
中華民國選舉史上首位被罷免成功的縣市首長是誰:  韓國瑜
罷免有效聯署書達到多少?:  37.7萬
韓國瑜在哪天正式解職:  6月12日
韓國瑜幾歲被轉入放牛班:  國中三年級
韓國瑜每天上課一直看誰小腿:  前排女同學
韓國瑜的碩士論文名稱?:  《從中共「對臺統戰」策略看兩航談判》
韓國瑜退伍前一年考上什麼:  東吳大學英國語文學系
韓國瑜為什麼成績一落千丈:  青春期懵懂，每天上課一直看前排女同學的小腿
韓國瑜擔任誰的助理:  馮定國


In [37]:
content = '賈伯斯在1970年代末與蘋果公司另一始創人史蒂夫·沃茲尼克及首任投資者邁克·馬庫拉協同其他人設計、\
開發及銷售Apple II系列。在1980年代初，賈伯斯是最早看到全錄帕洛奧圖中心（Xerox PARC）的滑鼠驅動圖形用戶介面的商業潛力，\
並將其應用於Apple Lisa及一年後的麥金塔電腦。1985年，在董事會的鬥爭失勢後，賈伯斯離開蘋果公司及成立了NeXT公司(一間電腦\
平台開發公司，專門從事高等教育及商業市場)在1986年，他收購了盧卡斯影業的電腦繪圖部門，成立了皮克斯（Pixar）。\
他被譽為《玩具總動員》（1995年）的執行製片人。他一直擔任皮克斯動畫的執行長並持有50.1%的股份，\
直到公司在2006年被華特迪士尼公司收購，此項收購使賈伯斯成為迪士尼公司的最大個人股東（有7.4%的股份）及董事會成員。\
在1996年，蘋果公司董事會決議買下NeXT公司，把賈伯斯帶回他參與創立，卻正在垂死邊緣的蘋果公司擔任臨時CEO。\
他在2000年起成為正式CEO，帶領蘋果輝煌的iPod、iPhone、iPad時代的到來。從2003年10月起，賈伯斯與胰腺神經內分泌腫瘤奮戰了8年，\
最終於2011年8月辭任執行長一職，在他第3次病假期間，賈伯斯當選為蘋果公司的董事長。'


question = ['NeXT公司是誰創造的','NeXT公司是在哪年成立','誰決議買下NeXT公司','誰成立了皮克斯'
            ,'誰是《玩具總動員》的執行製片人','皮克斯動畫在哪年被華特迪士尼公司收購']


for q in question:
    print(q + ':  '+ myfriend.process(content,q))



NeXT公司是誰創造的:  賈伯斯
NeXT公司是在哪年成立:  1985年
誰決議買下NeXT公司:  董事會
誰成立了皮克斯:  賈伯斯
誰是《玩具總動員》的執行製片人:  賈伯斯
皮克斯動畫在哪年被華特迪士尼公司收購:  2006年


In [39]:
content = f'叡揚資訊近二十幾年發展商業智慧及資料倉儲系統的建置經驗中，\
提供了以數據分析及 AI 輔助的智慧醫療與智慧零售的解決方案，\
更因此獲得了2019 APICTA Awards 大數據應用銀牌的肯定。\
APICTA 亞太資通訊聯盟大賽，這個舉辦已 18 屆，\
每年超過 200 個各國企業／團隊參賽的競賽，是亞太區最具影響力的資通訊科技競賽，\
更有亞太資通訊科技奧斯卡獎之稱。感謝評審對叡揚大數據應用解決方案肯定，\
對台灣的軟實力更是一劑強心針讓台灣與叡揚在國際的舞台繼續發光發熱。'

question = ['APICTA 亞太資通訊聯盟大賽每ㄋㄧㄞˊ']

for q in question:
    print(q + ':  '+ myfriend.process(content,q))

叡揚資訊獲得了甚麼獎項:  抱歉 這題有點難耶


# F1

In [6]:
from sklearn.metrics import f1_score

In [11]:
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 1, 2, 0, 1, 2]
f1_score(y_true, y_pred, average='macro')

1.0

In [1]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [5]:
y_true = np.array(['cat', 'dog', 'pig', 'cat', 'dog', 'pig'])
y_pred = np.array(['cat', 'pig', 'dog', 'cat', 'cat', 'dog'])
precision_recall_fscore_support(y_true, y_pred, average='macro')

(0.2222222222222222, 0.3333333333333333, 0.26666666666666666, None)